# Getting data from the web:  Scraping

Our trusty friend Pandas can read data directly from a web link.

We read the dataset into a dataframe without actually having the file in our folder!

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://corgis-edu.github.io/corgis/datasets/csv/classics/classics.csv')

In [ ]:
df.sample()

In [ ]:
df.sort_values(by='metadata.downloads',ascending=True)[-20:].plot.barh(x='bibliography.title',y='metadata.downloads')

---

## requests

The above code is very handy, but what if we simply want to read content that is on the page rather than in a readily available file?

Go to the [Classics CSV File](https://corgis-edu.github.io/corgis/csv/classics/) webpage and use your Browser's Inspector to look at the HTML for the page.  This will show HTML we discussed very briefly during Week 1.

We are going to get the entire web page using "requests" ([documentation](https://docs.python-requests.org/en/latest/)).  
* "Requests is an elegant and simple HTTP library for Python, built for human beings."

In [ ]:
import requests

In [ ]:
response = requests.get('https://corgis-edu.github.io/corgis/csv/classics/')

In [ ]:
response

"Responses" are numerical codes that indicate whether a specific HTTP request has been successfully completed (See [HTTP code list](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status))

You may have run into a couple of these on other sites, or even while trying to login to this JupyterHub!

In [ ]:
# Note that this won't actually get the csv file

response = requests.get('https://corgis-edu.github.io/corgis/csv/classics/classics.csv')

In [ ]:
response

Try finding the above URL in your browser:  https://corgis-edu.github.io/corgis/csv/classics/classics.csv

In [ ]:
response = requests.get('https://corgis-edu.github.io/corgis/csv/classics/')
print(response)

In [ ]:
# The html of our desired corgis page:
response.text

In [ ]:
print(response.text)

It is possible to search through the web tags to find what it may be that you want to search for:

In [ ]:
# Save all the html in a string variable
html_string = response.text

# Use BeautifulSoup to create a new object that will allow you to search for HTML tags
document = BeautifulSoup(html_string, "html.parser")

# This "document" variable is an object that has a "find" method
document.find('a')

What is `<a href="...`?

-> This is an HTML tag.... so what are HTML "tags"?

HTML: Hyper-Text Markup Language

HTML uses "tags" to classify different elements, for example:
* `<h1>...</h1>`: a large header
* `<img src="...">`: an image
* `<a href="...">Deep Space Nine</a>`: a link

Let's look at a simpler website:
http://static.decontextualize.com/kittens.html

In [ ]:
response = requests.get('http://static.decontextualize.com/kittens.html')

In [ ]:
print(response.text)

Here the tag examples are:
* `<h1>Kittens and the TV Shows They Love</h1>`: a large header
* `<img src="http://placekitten.com/120/120">`: an image
* `<a href="http://www.imdb.com/title/tt0106145/">Deep Space Nine</a>`: a link

And you'll see additional tags:
* `<ul>`: unordered list
* `<li>`: list item
* `<head>` and `<body>`: like header information and the body of a document
* `<div>`: section of the document

There's a lot to learn about HTML, but this is mainly to show you examples of tags.  BeautifulSoup will let you parse HTML documents based on these tags.

---

Fun aside:  you can use the IPython library to visualize HTML right inside the Jupyter notebook.

In [ ]:
from IPython.core.display import HTML
HTML('<img src="http://placekitten.com/110/110">')

In [ ]:
%%HTML
<img src="http://placekitten.com/110/110">

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('Awf45u6zrP0')

---

Ok, enough cat silliness...

---

Back to our literary classics.

In [ ]:
response = requests.get('https://corgis-edu.github.io/corgis/csv/classics/')
html_string = response.text
document = BeautifulSoup(html_string, "html.parser")

In [ ]:
# We can look for the first link:

document.find('a')

In [ ]:
# We can search for all the links on the page with:

document.find_all('a')

This allows us now to find the download link for the csv file.

# Two examples for practice

### Getting the script of Coco from IMSDB

In [ ]:
import requests

In [ ]:
response = requests.get('https://imsdb.com/scripts/Coco.html')

In [ ]:
response

In [ ]:
response.text

Woah, too much!

We break it down with BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
html_string = response.text
document = BeautifulSoup(html_string, "html.parser")

In [ ]:
document

`document` itself is still the original HTML

In [ ]:
type(document)

In [ ]:
document.find('pre')

There is still a lot of formatting to work through, but we have now found the screenplay text.

In [ ]:
print(document.find('pre').text)

### Grabbing data from GitHub

Let's look at the repository lists of the Pandas-relevant organization

In [ ]:
import requests

In [ ]:
response = requests.get('https://github.com/orgs/pandas-dev/repositories')

In [ ]:
response

In [ ]:
html_string = response.text

In [ ]:
html_string

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
document = BeautifulSoup(html_string, "html.parser")

In [ ]:
document

In [ ]:
document.find("a")

In [ ]:
document.find("a").attrs

In [ ]:
document.find("a", attrs={'itemprop':'name codeRepository'})

In [ ]:
document.find_all("a", attrs={'itemprop':'name codeRepository'})

In [ ]:
for i in document.find_all("a", attrs={'itemprop':'name codeRepository'}):
    print(i.text.strip())

In [ ]:
for i in document.find_all("a", attrs={'itemprop':'name codeRepository'}):
    print(i.attrs)

In [ ]:
for i in document.find_all("a", attrs={'itemprop':'name codeRepository'}):
    print(i.text.strip() + ' : accessible at http://github.com' + i.attrs['href'])

### Ok, maybe three examples
The NYTimes has an organizational account on GitHub too:

In [ ]:
response = requests.get('https://github.com/orgs/nytimes/repositories')
html_string = response.text
document = BeautifulSoup(html_string, "html.parser")
for i in document.find_all("a", attrs={'itemprop':'name codeRepository'}):
    print(i.text.strip() + ' : accessible at http://github.com' + i.attrs['href'])

Let's say we get interested in looking at their covid-19-data repo.  Visit that repo page by clicking on the link above.

And we can try to directly import info from their us-states.csv file.

In [ ]:
# This will fail:
# df = pd.read_csv('https://github.com/nytimes/covid-19-data/blob/master/rolling-averages/us-states.csv')

# must replace "blob" with "master" -> note the download link on the github page
df = pd.read_csv('https://github.com/nytimes/covid-19-data/raw/master/rolling-averages/us-states.csv')

In [ ]:
df.head()

In [ ]:
df.groupby('state')['cases'].sum()

In [ ]:
df.groupby('state')['cases'].sum().sort_values().plot.barh(figsize=(8,10))